# Goal

* Primer design for clade of interest

# Var

In [2]:
base_dir = '/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/'
clade = 'Prevotella'
domain = 'Bacteria'
taxid = 838

# Init

In [5]:
library(dplyr)
library(tidyr)
library(data.table)
library(tidytable)
library(ggplot2)
library(LeyLabRMisc)

In [6]:
df.dims()
work_dir = file.path(base_dir, clade)
make_dir(work_dir)

Directory already exists: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3//Prevotella 


# Genome download

* Downloading genomes from NCBI

```
OUTDIR=/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted//Prevotella/
mkdir -p $OUTDIR
ncbi-genome-download -p 12 -s genbank -F fasta --genera /Prevotella -o $OUTDIR bacteria
```

# Genome quality

* Filtering genomes by quality

In [9]:
D = file.path(base_dir, clade, 'genbank')
files = list_files(D, '.fna.gz')
samps = data.frame(Name = files %>% as.character %>% basename,
                   Fasta = files,
                   Domain = domain,
                   Taxid = taxid) %>%
    mutate(Name = gsub('\\.fna\\.gz$', '', Name),
           Fasta = gsub('/+', '/', Fasta))
samps

# writing file
outfile = file.path(D, 'samples.txt')
write_table(samps, outfile)

Name,Fasta,Domain,Taxid
<chr>,<chr>,<fct>,<dbl>
GCA_000025925.1_ASM2592v1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Prevotella/genbank/bacteria/GCA_000025925.1/GCA_000025925.1_ASM2592v1_genomic.fna.gz,Bacteria,838
GCA_000142965.1_ASM14296v1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Prevotella/genbank/bacteria/GCA_000142965.1/GCA_000142965.1_ASM14296v1_genomic.fna.gz,Bacteria,838
⋮,⋮,⋮,⋮
GCA_902399905.1_UHGG_MGYG-HGUT-03697_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Prevotella/genbank/bacteria/GCA_902399905.1/GCA_902399905.1_UHGG_MGYG-HGUT-03697_genomic.fna.gz,Bacteria,838
GCA_902794455.1_Rumen_uncultured_genome_RUG13974_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Prevotella/genbank/bacteria/GCA_902794455.1/GCA_902794455.1_Rumen_uncultured_genome_RUG13974_genomic.fna.gz,Bacteria,838


File written: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3//Prevotella/genbank/samples.txt 


### LLG

#### Config

In [10]:
cat_file(file.path(work_dir, 'LLG', 'config.yaml'))

# table with genome --> fasta_file information
samples_file: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Prevotella/genbank/samples.txt

# output location
output_dir: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Prevotella/LLG/

# temporary file directory (your username will be added automatically)
tmp_dir: /ebio/abt3_scratch/

# batch processing of genomes for certain steps
## increase to better parallelize
batches: 2 

# Domain of genomes ('Archaea' or 'Bacteria)
## Use "Skip" if provided as a "Domain" column in the genome table
Domain: Skip

# software parameters
# Use "Skip" to skip any of these steps. If no params for rule, use ""
# dRep MAGs are not further analyzed, but you can de-rep & then use the de-rep genome table as input.
params:
  ionice: -c 3
  # assembly assessment
  seqkit: ""
  quast: Skip #""
  multiqc_on_quast: "" 
  checkm: ""
  # de-replication (requires checkm)
  drep: -comp 9

#### Run

```
(snakemake) @ rick:/ebio/abt3_projects/software/dev/ll_pipelines/llg
$ screen -L -S llg ./snakemake_sge.sh /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Prevotella/LLG/config.yaml 20 -F
```

### Samples table of high-quality genomes

In [36]:
# checkM summary
checkm = file.path(work_dir, 'LLG', 'checkM', 'checkm_qa_summary.tsv') %>%
    read.delim(sep='\t') 
checkm

Bin.Id,Marker.lineage,X..genomes,X..markers,X..marker.sets,Completeness,Contamination,Strain.heterogeneity,Genome.size..bp.,X..ambiguous.bases,⋯,X0,X1,X2,X3,X4,X5.,assembly.Id,assembler.Id,taxon.Id,File
<fct>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<fct>,<fct>,<lgl>,<fct>
GCA_000025925.1_ASM2592v1_genomic,o__Bacteroidales (UID2716),92,486,270,99.63,0.37,100,3619559,0,⋯,1,484,1,0,0,0,|ebio|abt3_projects|software|dev|ll_pipelines|llprimer|experiments|HMP_most-wanted|Prevotella|LLG_output|checkM|1|checkm|markers_qa_summary.tsv.1,markers_qa_summary.tsv.1,NA,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Prevotella/LLG_output/checkM/1/checkm/markers_qa_summary.tsv.1
GCA_000144405.1_ASM14440v1_genomic,g__Prevotella (UID2724),55,567,296,99.32,0.00,0,3168282,0,⋯,2,565,0,0,0,0,|ebio|abt3_projects|software|dev|ll_pipelines|llprimer|experiments|HMP_most-wanted|Prevotella|LLG_output|checkM|1|checkm|markers_qa_summary.tsv.2,markers_qa_summary.tsv.2,NA,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Prevotella/LLG_output/checkM/1/checkm/markers_qa_summary.tsv.2
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
GCA_902386825.1_UHGG_MGYG-HGUT-02412_genomic,o__Bacteroidales (UID2657),160,492,269,97.58,0.00,0.00,3475353,3078,⋯,7,485,0,0,0,0,|ebio|abt3_projects|software|dev|ll_pipelines|llprimer|experiments|HMP_most-wanted|Prevotella|LLG_output|checkM|2|checkm|markers_qa_summary.tsv.459,markers_qa_summary.tsv.459,NA,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Prevotella/LLG_output/checkM/2/checkm/markers_qa_summary.tsv.459
GCA_902399905.1_UHGG_MGYG-HGUT-03697_genomic,g__Prevotella (UID2724),55,567,296,98.14,1.46,28.57,3900702,818,⋯,7,553,7,0,0,0,|ebio|abt3_projects|software|dev|ll_pipelines|llprimer|experiments|HMP_most-wanted|Prevotella|LLG_output|checkM|2|checkm|markers_qa_summary.tsv.460,markers_qa_summary.tsv.460,NA,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Prevotella/LLG_output/checkM/2/checkm/markers_qa_summary.tsv.460


In [37]:
# dRep summary
drep = file.path(work_dir, 'LLG', 'drep', 'checkm_markers_qa_summary.tsv') %>%
    read.delim(sep='\t') %>%
    mutate(Bin.Id = gsub('.+/', '', genome),
           Bin.Id = gsub('\\.fna$', '', Bin.Id))
drep

genome,completeness,contamination,Bin.Id
<fct>,<dbl>,<dbl>,<chr>
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000025925.1_ASM2592v1_genomic.fna,99.63,0.37,GCA_000025925.1_ASM2592v1_genomic
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000144405.1_ASM14440v1_genomic.fna,99.32,0.00,GCA_000144405.1_ASM14440v1_genomic
⋮,⋮,⋮,⋮
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_902386825.1_UHGG_MGYG-HGUT-02412_genomic.fna,97.58,0.00,GCA_902386825.1_UHGG_MGYG-HGUT-02412_genomic
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_902399905.1_UHGG_MGYG-HGUT-03697_genomic.fna,98.14,1.46,GCA_902399905.1_UHGG_MGYG-HGUT-03697_genomic


In [38]:
# de-replicated genomes
drep_gen = file.path(work_dir, 'LLG', 'drep', 'dereplicated_genomes.tsv') %>%
    read.delim(sep='\t')
drep_gen

Name,Fasta
<fct>,<fct>
GCA_002342585.1_ASM234258v1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Prevotella/LLG/drep/drep/dereplicated_genomes/GCA_002342585.1_ASM234258v1_genomic.fna
GCA_002449845.1_ASM244984v1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Prevotella/LLG/drep/drep/dereplicated_genomes/GCA_002449845.1_ASM244984v1_genomic.fna
⋮,⋮
GCA_000437495.1_MGS485_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Prevotella/LLG/drep/drep/dereplicated_genomes/GCA_000437495.1_MGS485_genomic.fna
GCA_002297545.1_ASM229754v1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Prevotella/LLG/drep/drep/dereplicated_genomes/GCA_002297545.1_ASM229754v1_genomic.fna


In [39]:
# GTDBTk summary
tax = file.path(work_dir, 'LLG', 'gtdbtk', 'gtdbtk_bac_summary.tsv') %>%
    read.delim(, sep='\t') %>%
    separate(classification, 
             c('Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species'),
             sep=';') %>%
    select(-note, -classification_method, -pplacer_taxonomy,
           -other_related_references.genome_id.species_name.radius.ANI.AF.)
tax

user_genome,Domain,Phylum,Class,Order,Family,Genus,Species,fastani_reference,fastani_reference_radius,⋯,fastani_af,closest_placement_reference,closest_placement_radius,closest_placement_taxonomy,closest_placement_ani,closest_placement_af,msa_percent,translation_table,red_value,warnings
<fct>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<int>,<fct>,<fct>
GCA_000025925.1_ASM2592v1_genomic,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Bacteroidales,f__Bacteroidaceae,g__Prevotella,s__Prevotella ruminicola,GCF_000025925.1,95.0,⋯,1.0,GCF_000025925.1,95.0,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Prevotella;s__Prevotella ruminicola,100.0,1.0,96.11,11,N/A,N/A
GCA_000144405.1_ASM14440v1_genomic,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Bacteroidales,f__Bacteroidaceae,g__Prevotella,s__Prevotella melaninogenica,GCF_000144405.1,95.0,⋯,1.0,GCF_000144405.1,95.0,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Prevotella;s__Prevotella melaninogenica,100.0,1.0,96.83,11,N/A,N/A
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
GCA_902386825.1_UHGG_MGYG-HGUT-02412_genomic,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Bacteroidales,f__Bacteroidaceae,g__Prevotellamassilia,s__Prevotellamassilia timonensis,GCF_900106785.1,95.0,⋯,0.99,GCF_900106785.1,95.0,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Prevotellamassilia;s__Prevotellamassilia timonensis,100.0,0.99,96.09,11,N/A,N/A
GCA_902399905.1_UHGG_MGYG-HGUT-03697_genomic,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Bacteroidales,f__Bacteroidaceae,g__Prevotella,s__Prevotella copri_A,GCF_002224675.1,95.0,⋯,0.75,GCF_002224675.1,95.0,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Prevotella;s__Prevotella copri_A,96.12,0.75,96.07,11,N/A,N/A


In [40]:
# checking overlap
cat('-- drep --\n')
overlap(basename(as.character(drep_gen$Fasta)), 
        basename(as.character(drep$genome)))
cat('-- checkm --\n')
overlap(drep$Bin.Id, checkm$Bin.Id)
cat('-- gtdbtk --\n')
overlap(drep$Bin.Id, tax$user_genome)

-- drep --
intersect(x,y): 458 
setdiff(x,y): 0 
setdiff(y,x): 463 
union(x,y): 921 
-- checkm --
intersect(x,y): 921 
setdiff(x,y): 0 
setdiff(y,x): 0 
union(x,y): 921 
-- gtdbtk --
intersect(x,y): 921 
setdiff(x,y): 0 
setdiff(y,x): 0 
union(x,y): 921 


In [41]:
# joining based on Bin.Id
drep = drep %>%
    inner_join(checkm, c('Bin.Id')) %>%
    mutate(GEN = genome %>% as.character %>% basename) %>%
    inner_join(drep_gen %>% mutate(GEN = Fasta %>% as.character %>% basename),
               by=c('GEN')) %>%
    inner_join(tax, c('Bin.Id'='user_genome')) #%>%
drep$genome %>% unique_n('genomes')
drep

No. of unique genomes: 458 


genome,completeness,contamination,Bin.Id,Marker.lineage,X..genomes,X..markers,X..marker.sets,Completeness,Contamination,⋯,fastani_af,closest_placement_reference,closest_placement_radius,closest_placement_taxonomy,closest_placement_ani,closest_placement_af,msa_percent,translation_table,red_value,warnings
<fct>,<dbl>,<dbl>,<chr>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<int>,<fct>,<fct>
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000025925.1_ASM2592v1_genomic.fna,99.63,0.37,GCA_000025925.1_ASM2592v1_genomic,o__Bacteroidales (UID2716),92,486,270,99.63,0.37,⋯,1.0,GCF_000025925.1,95.0,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Prevotella;s__Prevotella ruminicola,100.0,1.0,96.11,11,N/A,N/A
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000144405.1_ASM14440v1_genomic.fna,99.32,0.00,GCA_000144405.1_ASM14440v1_genomic,g__Prevotella (UID2724),55,567,296,99.32,0.00,⋯,1.0,GCF_000144405.1,95.0,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Prevotella;s__Prevotella melaninogenica,100.0,1.0,96.83,11,N/A,N/A
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_900607215.1_PRJEB27263_genomic.fna,98.65,0.16,GCA_900607215.1_PRJEB27263_genomic,g__Prevotella (UID2724),55,567,296,98.65,0.16,⋯,1.0,GCF_900607215.1,95.0,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Prevotella;s__Prevotella sp900607215,100.0,1.0,96.09,11,N/A,N/A
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_900637655.1_52295_B01_genomic.fna,99.32,0.34,GCA_900637655.1_52295_B01_genomic,g__Prevotella (UID2724),55,567,296,99.32,0.34,⋯,1.0,GCF_900637655.1,95.0,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Prevotella;s__Prevotella oris,100.0,1.0,98.00,11,N/A,N/A


In [42]:
# filtering by quality
hq_genomes = drep %>%
    filter(completeness >= 99,
           contamination < 5,
           Strain.heterogeneity < 50,
           X..contigs <= 100,
           Mean.contig.length..bp. >= 20000,
           N50..contigs. >= 100000)#,
#           X..predicted.genes >= 2000,
#           X..predicted.genes <= 3000)
hq_genomes$genome %>% unique_n('genomes')
hq_genomes

No. of unique genomes: 91 


genome,completeness,contamination,Bin.Id,Marker.lineage,X..genomes,X..markers,X..marker.sets,Completeness,Contamination,⋯,fastani_af,closest_placement_reference,closest_placement_radius,closest_placement_taxonomy,closest_placement_ani,closest_placement_af,msa_percent,translation_table,red_value,warnings
<fct>,<dbl>,<dbl>,<chr>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<int>,<fct>,<fct>
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000144405.1_ASM14440v1_genomic.fna,99.32,0,GCA_000144405.1_ASM14440v1_genomic,g__Prevotella (UID2724),55,567,296,99.32,0,⋯,1.0,GCF_000144405.1,95.0,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Prevotella;s__Prevotella melaninogenica,100.0,1.0,96.83,11,N/A,N/A
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000162915.1_ASM16291v1_genomic.fna,99.32,0,GCA_000162915.1_ASM16291v1_genomic,g__Prevotella (UID2724),55,567,296,99.32,0,⋯,0.86,GCF_900637655.1,95.0,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Prevotella;s__Prevotella oris,97.06,0.86,97.20,11,N/A,N/A
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_900454835.1_52295_A01_genomic.fna,99.21,0,GCA_900454835.1_52295_A01_genomic,g__Prevotella (UID2724),55,567,296,99.21,0,⋯,1.0,GCF_000613585.1,95.0,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Prevotella;s__Prevotella denticola,99.97,1.0,96.65,11,N/A,N/A
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_900454935.1_57800_C01_genomic.fna,99.66,0,GCA_900454935.1_57800_C01_genomic,g__Prevotella (UID2724),55,567,296,99.66,0,⋯,0.88,GCF_000220255.1,95.0,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Prevotella;s__Prevotella pallens,97.92,0.88,97.00,11,N/A,N/A


In [43]:
# summary
hq_genomes %>% summary_x('completeness', completeness)
hq_genomes %>% summary_x('comtamination', contamination)
hq_genomes %>% summary_x('hetero', Strain.heterogeneity)
hq_genomes %>% summary_x('no. of contigs', X..contigs)
hq_genomes %>% summary_x('mean contig length', Mean.contig.length..bp.)
hq_genomes %>% summary_x('no. of genes', X..predicted.genes)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
completeness,99.14,99.29,99.32,99.38626,99.605,99.66,0.198,0.081


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
comtamination,0,0,0.08,0.2910989,0.49,2.04,0.786,0.321


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
hetero,0,0,0,0.3662637,0,33.33,13.578,5.543


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
no. of contigs,1,3.5,22,31.63736,50,98,36.111,14.742


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
mean contig length,30476,67069.5,135673,499684.3,885175.5,2937589,1116017,455611.9


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
no. of genes,2068,2455.5,2591,2649.055,2808,3455,458.335,187.115


In [44]:
# summarizing the taxonomy
df.dims(20)
hq_genomes %>%
    group_by(Family, Genus) %>%
    summarize(n_genomes = n(), .groups='drop')
df.dims()

Family,Genus,n_genomes
<chr>,<chr>,<int>
f__Bacteroidaceae,g__Prevotella,91


In [45]:
# writing samples table for LLPRIMER
outfile = file.path(work_dir, 'samples_genomes_hq.txt')
hq_genomes %>%
    select(Bin.Id, Fasta) %>%
    rename('Taxon' = Bin.Id) %>%
    mutate(Taxon = gsub('_chromosome.+', '', Taxon),
           Taxon = gsub('_bin_.+', '', Taxon),
           Taxon = gsub('_genomic', '', Taxon),
           Taxon = gsub('_annotated_assembly', '', Taxon),
           Taxid = taxid,
           Domain = domain) %>%
    write_table(outfile)

File written: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3//Prevotella/samples_genomes_hq.txt 


# Primer design

### Config

In [35]:
F = file.path(work_dir, 'primers', 'config.yaml')
cat_file(F)

#-- I/O --#
samples_file: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Prevotella/samples_genomes_hq.txt

# output location
output_dir: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Prevotella/primers/

# temporary file directory (your username will be added automatically)
tmp_dir: /ebio/abt3_scratch/

#-- software parameters --#
# See the README for a description
params:
  ionice: -c 3
  cgp:
    prokka: ""
    mmseqs:
      method: cluster    # or linclust (faster)
      run: --min-seq-id 0.8 -c 0.8
    vsearch: --id 0.94
    core_genes:
      cds: --perc-genomes-cds 100 --copies-per-genome-cds 1 --max-clusters-cds 500
      rRNA: --perc-genomes-rrna 100 --copies-per-genome-rrna 10 --max-clusters-rrna 500
    align:
      method: linsi
      params: --auto --maxiterate 1000
    primer3:
      consensus: --consensus-threshold 0.34
      number: --num-raw-primers 1000 --num-final-primers 10
      size:

### Run

```
(snakemake) @ rick:/ebio/abt3_projects/software/dev/ll_pipelines/llprimer
$ screen -L -S llprimer-Prev ./snakemake_sge.sh /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Prevotella/primers/config.yaml 30 -F --notemp
```

## Summary

### Primers

In [62]:
primer_info = read.delim(file.path(work_dir, 'primers', 'cgp', 'primers_final_info.tsv'), sep='\t')
primer_info %>% unique_n('primers', primer_set)
primer_info

No. of unique primers: 1 


gene_type,cluster_id,primer_set,amplicon_size_consensus,amplicon_size_avg,amplicon_size_sd,primer_id,primer_type,sequence,length,⋯,position_start,position_end,Tm_avg,Tm_sd,GC_avg,GC_sd,hairpin_avg,hairpin_sd,homodimer_avg,homodimer_sd
<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<int>,⋯,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
cds,8,478,165,165,0,478f,PRIMER_LEFT,ATYAAGWGYCGYACNAARGG,20,⋯,474,494,57.34402,2.984923,47.5,5.590170,12.32349,18.38053,-5.195385,18.64899
cds,8,478,165,165,0,478r,PRIMER_RIGHT,NACRTTRCGGAAYTCYTGRT,20,⋯,619,639,57.13081,2.800376,45.0,6.123724,4.17719,11.05181,2.801095,21.40866


### Gene clusters

In [48]:
# target gene info
gene_annot = read.delim(file.path(work_dir, 'primers', 'cgp', 'core_clusters_info.tsv'), 
                        sep='\t') %>%
    semi_join(primer_info, c('cluster_id')) 
gene_annot %>% distinct(gene_type, cluster_id) %>% nrow %>% cat('No. of clusters', ., '\n')
gene_annot

No. of clusters 1 


gene_type,cluster_id,seq_uuid,seq_orig_name,contig_id,taxon,start,end,score,strand,annotation,cluster_name,clust_id
<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>
cds,8,3404d3ff12a64e38aca73b6aa31fae58,JIKNMOCF_01515,CP002123.1,GCA_000144405.1_ASM14440v1,97075,98859,.,-,Polyribonucleotide nucleotidyltransferase,6ab20e4d4d0f406a8d66f92f6486129a,8
cds,8,f220b5952cb749519bec8d6eeac845fe,NBHLGDOI_01200,GG703886.1,GCA_000162915.1_ASM16291v1,336896,338677,.,-,30S ribosomal protein S1,6ab20e4d4d0f406a8d66f92f6486129a,8
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
cds,8,a48a661ccd0a47d59993e36b95147fbc,AFOLKJKN_00363,UGTM01000002.1,GCA_900454835.1_52295_A01,451758,453539,.,-,Polyribonucleotide nucleotidyltransferase,6ab20e4d4d0f406a8d66f92f6486129a,8
cds,8,299907c79c5a446fa61edd59ef341b1c,GDKFNPGA_00758,UGTP01000002.1,GCA_900454935.1_57800_C01,294357,296132,.,+,30S ribosomal protein S1,6ab20e4d4d0f406a8d66f92f6486129a,8


### Gene cluster annotations

In [50]:
# non-target cds gene annotations
gene_nontarget = read.delim(file.path(work_dir, 'primers', 'cgp', 'nontarget', 'cds_blastx.tsv'), 
                        sep='\t') %>%
    semi_join(primer_info, c('cluster_id')) 
gene_nontarget

cluster_id,query,subject,subject_name,pident,length,mismatch,qstart,qend,sstart,send,evalue,slen,qlen,sscinames,staxids,pident_rank
<int>,<fct>,<fct>,<fct>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<fct>,<fct>,<int>
8,6ab20e4d4d0f406a8d66f92f6486129a,SFW41323.1,SSU ribosomal protein S1P [Prevotellaceae bacterium HUN156],90.674,579,53,1,1734,1,579,0,592,1779,Prevotellaceae bacterium HUN156,1520830,2
8,6ab20e4d4d0f406a8d66f92f6486129a,SNU04056.1,SSU ribosomal protein S1P [Prevotellaceae bacterium MN60],90.846,579,52,1,1734,1,579,0,592,1779,Prevotellaceae bacterium MN60,1945887,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
8,6ab20e4d4d0f406a8d66f92f6486129a,WP_044264685.1,MULTISPECIES: 30S ribosomal protein S1 [Bacteroides],80.517,580,106,16,1734,5,584,0,598,1779,Bacteroides;Bacteroides oleiciplenus;Bacteroides stercorirosoris;Bacteroides timonensis,816;626931;871324;1470345,25
8,6ab20e4d4d0f406a8d66f92f6486129a,WP_007757364.1,MULTISPECIES: 30S ribosomal protein S1 [Bacteroides],80.446,583,104,16,1734,5,587,0,600,1779,Bacteroides;Bacteroides finegoldii;Bacteroides finegoldii DSM 17565;Bacteroides sp. NSJ-39,816;338188;483215;2763019,26


### Cluster taxonomic uniqueness

In [51]:
# closest seqID
gene_nontarget %>%
    filter(pident_rank == 1) %>%
    .$pident %>% summary_x('seq-ID to closest non-target')

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
seq-ID to closest non-target,90.846,90.846,90.846,90.846,90.846,90.846,0,0


### Primer quality

In [52]:
# summary
primer_info %>% unique_n('primer sets', primer_set)
primer_info %>% distinct(gene_type, cluster_id) %>% nrow %>% cat('No. of unique clusters:', ., '\n')

No. of unique primer sets: 1 
No. of unique clusters: 1 


In [53]:
# primer per cluster
primer_info %>%
    distinct(cluster_id, primer_set) %>%
    group_by(cluster_id) %>%
    summarize(n_primer_sets = n(), .groups='drop') %>%
    .$n_primer_sets %>% summary_x('primer sets per cluster')

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
primer sets per cluster,1,1,1,1,1,1,0,0


In [54]:
# primer quality
primer_info %>% filter(primer_type == 'PRIMER_LEFT') %>% .$amplicon_size_avg %>% summary_x('mean amplicon size')
primer_info %>% .$degeneracy %>% summary_x('degeneracy')
primer_info %>% .$degeneracy_3prime %>% summary_x('degeneracy (3-prime)')

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
mean amplicon size,165,165,165,165,165,165,0,0


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
degeneracy,128,128,128,128,128,128,0,0


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
degeneracy (3-prime),2,2.5,3,3,3.5,4,0.707,0.289


In [55]:
# adding non-target pident info
tmp1 = gene_nontarget %>%
    filter(pident_rank == 1) %>%
    distinct(cluster_id, pident, sscinames) %>%
    mutate(gene_type = 'cds')

primer_info_j = primer_info %>%
    inner_join(tmp1, c('gene_type', 'cluster_id'))
primer_info_j

gene_type,cluster_id,primer_set,amplicon_size_consensus,amplicon_size_avg,amplicon_size_sd,primer_id,primer_type,sequence,length,⋯,Tm_avg,Tm_sd,GC_avg,GC_sd,hairpin_avg,hairpin_sd,homodimer_avg,homodimer_sd,pident,sscinames
<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
cds,8,478,165,165,0,478f,PRIMER_LEFT,ATYAAGWGYCGYACNAARGG,20,⋯,57.34402,2.984923,47.5,5.590170,12.32349,18.38053,-5.195385,18.64899,90.846,Prevotellaceae bacterium MN60
cds,8,478,165,165,0,478r,PRIMER_RIGHT,NACRTTRCGGAAYTCYTGRT,20,⋯,57.13081,2.800376,45.0,6.123724,4.17719,11.05181,2.801095,21.40866,90.846,Prevotellaceae bacterium MN60


In [56]:
# arrange by "best" primers
primer_info_jf = primer_info_j %>%
    group_by(primer_set) %>%
    mutate(max_degeneracy_3prime = max(degeneracy_3prime),  # max of [Fwd,Rev]
           max_degeneracy = max(degeneracy)) %>%
    ungroup() %>%
    arrange(pident, max_degeneracy_3prime, max_degeneracy) %>%
    head(n=30)

df.dims(30,40)
primer_info_jf
df.dims()

gene_type,cluster_id,primer_set,amplicon_size_consensus,amplicon_size_avg,amplicon_size_sd,primer_id,primer_type,sequence,length,degeneracy,degeneracy_3prime,position_start,position_end,Tm_avg,Tm_sd,GC_avg,GC_sd,hairpin_avg,hairpin_sd,homodimer_avg,homodimer_sd,pident,sscinames,max_degeneracy_3prime,max_degeneracy
<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<int>,<int>
cds,8,478,165,165,0,478f,PRIMER_LEFT,ATYAAGWGYCGYACNAARGG,20,128,2,474,494,57.34402,2.984923,47.5,5.590170,12.32349,18.38053,-5.195385,18.64899,90.846,Prevotellaceae bacterium MN60,4,128
cds,8,478,165,165,0,478r,PRIMER_RIGHT,NACRTTRCGGAAYTCYTGRT,20,128,4,619,639,57.13081,2.800376,45.0,6.123724,4.17719,11.05181,2.801095,21.40866,90.846,Prevotellaceae bacterium MN60,4,128


#### w/ gene annotations

In [57]:
# joining
primer_info_jfj = primer_info_jf %>%
    left_join(gene_annot %>% distinct(gene_type, cluster_id, annotation),
              c('gene_type', 'cluster_id'))

df.dims(30,50)
primer_info_jfj
df.dims()

gene_type,cluster_id,primer_set,amplicon_size_consensus,amplicon_size_avg,amplicon_size_sd,primer_id,primer_type,sequence,length,degeneracy,degeneracy_3prime,position_start,position_end,Tm_avg,Tm_sd,GC_avg,GC_sd,hairpin_avg,hairpin_sd,homodimer_avg,homodimer_sd,pident,sscinames,max_degeneracy_3prime,max_degeneracy,annotation
<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<int>,<int>,<fct>
cds,8,478,165,165,0,478f,PRIMER_LEFT,ATYAAGWGYCGYACNAARGG,20,128,2,474,494,57.34402,2.984923,47.5,5.590170,12.32349,18.38053,-5.195385,18.64899,90.846,Prevotellaceae bacterium MN60,4,128,Polyribonucleotide nucleotidyltransferase
cds,8,478,165,165,0,478f,PRIMER_LEFT,ATYAAGWGYCGYACNAARGG,20,128,2,474,494,57.34402,2.984923,47.5,5.590170,12.32349,18.38053,-5.195385,18.64899,90.846,Prevotellaceae bacterium MN60,4,128,30S ribosomal protein S1
cds,8,478,165,165,0,478r,PRIMER_RIGHT,NACRTTRCGGAAYTCYTGRT,20,128,4,619,639,57.13081,2.800376,45.0,6.123724,4.17719,11.05181,2.801095,21.40866,90.846,Prevotellaceae bacterium MN60,4,128,Polyribonucleotide nucleotidyltransferase
cds,8,478,165,165,0,478r,PRIMER_RIGHT,NACRTTRCGGAAYTCYTGRT,20,128,4,619,639,57.13081,2.800376,45.0,6.123724,4.17719,11.05181,2.801095,21.40866,90.846,Prevotellaceae bacterium MN60,4,128,30S ribosomal protein S1


#### Writing table

In [58]:
F = file.path(work_dir, 'best_primers.tsv')
write_table(primer_info_jfj, F)

File written: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3//Prevotella/best_primers.tsv 


# sessionInfo

In [59]:
pipelineInfo('/ebio/abt3_projects/software/dev/ll_pipelines/llg/')

LLG
===

Ley Lab Genome analysis pipeline (LLG)

* Version: 0.2.4
* Authors:
  * Nick Youngblut <nyoungb2@gmail.com>
* Maintainers:
  * Nick Youngblut <nyoungb2@gmail.com>

--- conda envs ---
==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/gtdbtk.yaml <==
channels:
- conda-forge
- bioconda
dependencies:
- pigz
- conda-forge::networkx
- bioconda::gtdbtk

==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/checkm.yaml <==
channels:
- bioconda
dependencies:
- python=2.7
- pigz
- bioconda::prodigal
- bioconda::pplacer
- bioconda::checkm-genome

==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/quast.yaml <==
channels:
- conda-forge
- bioconda
dependencies:
- bioconda::seqkit
- bioconda::quast>=5.0.0

==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/sourmash.yaml <==
channels:
- conda-forge
- bioconda
dependencies:
- bioconda::sourmash=2.0.0a4

==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/fastqc.yaml <==
channels:
-

In [60]:
pipelineInfo('/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/')

LLPRIMER

Ley Lab Primer generation pipeline (LLPRIMER)

* Version: 0.3.0
* Authors:
  * Nick Youngblut <nyoungb2@gmail.com>
* Maintainers:
  * Nick Youngblut <nyoungb2@gmail.com>

--- conda envs ---
==> /ebio/abt3_projects/software/dev/ll_pipelines/llprimer//bin/envs/pdp.yaml <==
channels:
- conda-forge
- bioconda
dependencies:
- python=3.7
- intervaltree
- prodigal
- blast
- bedtools
- mafft
- mummer=3.23
- emboss
- primer3=1.1.4
- biopython<1.78
- pybedtools
- joblib
- tqdm
- openpyxl

==> /ebio/abt3_projects/software/dev/ll_pipelines/llprimer//bin/envs/genes.yaml <==
channels:
- bioconda
dependencies:
- pigz
- python=3
- numpy
- pyfaidx
- bioconda::seqkit
- bioconda::fasta-splitter
- bioconda::vsearch
- bioconda::mmseqs2
==> /ebio/abt3_projects/software/dev/ll_pipelines/llprimer//bin/envs/aln.yaml <==
channels:
- bioconda
- conda-forge
dependencies:
- pigz
- bioconda::revtrans
- bioconda::kalign3
- bioconda::mafft

==> /ebio/abt3_projects/software/dev/ll_pipelines/llprimer//bin/env

In [61]:
sessionInfo()

R version 3.6.3 (2020-02-29)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.5 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Georg_animal_feces/envs/tidyverse/lib/libopenblasp-r0.3.9.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] LeyLabRMisc_0.1.6 ggplot2_3.3.1     tidytable_0.5.1   data.table_1.12.8
[5] tidyr_1.1.0       dplyr_1.0.0      

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.4.6     magrittr_1.5     munsell_0.5.0    tidyselect_1.1.0
 [5] uuid_0.1-4       colorspace_1.4-1 R6_2.4.1  